# PNUD FLYNORTH

----
### Dato Primario

In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action="ignore", category=pd.core.common.SettingWithCopyWarning)

dir=r"D:\PROYECTOS 2024\PNUD\BOLIVAR\MADRID\8. Listado de coordenadas"
RA="17/09/2024"
Nombre="\\13430017-MADRID"


df=pd.read_csv(dir+"\\ptos.csv",sep=',',encoding='latin1')
df1=df[['Point Id','WGS84 Latitude [°]','WGS84 Longitude [°]','WGS84 Ellip. Height [m]','WGS84 Cartesian X [m]','WGS84 Cartesian Y [m]','WGS84 Cartesian Z [m]','Northing [m]','Easting [m]']]
df1["WGS84 Latitude [°]"] = df1["WGS84 Latitude [°]"].str.replace("° N", "")
df1["WGS84 Longitude [°]"] = df1["WGS84 Longitude [°]"].str.replace("° W", "")
df1["WGS84 Latitude [°]"]=df1['WGS84 Latitude [°]'].astype(float)
df1["WGS84 Longitude [°]"]=df1['WGS84 Longitude [°]'].astype(float)
df1["WGS84 Longitude [°]"] = df1["WGS84 Longitude [°]"]* -1
df1.to_csv(dir+"\\Data2024.csv",index=False)
### CAMBIO DE EPOCA
dfEpoca=df1[['Point Id','WGS84 Cartesian X [m]','WGS84 Cartesian Y [m]','WGS84 Cartesian Z [m]']]
dfEpoca['RA']=dfEpoca["RA"]=RA
dfEpoca['RE']=dfEpoca["RE"]="01/01/2018"
dfEpoca.to_csv(r"D:\PROYECTOS 2024\PNUD\Epoca.csv",index=False)

---
### Ondulacion
### Dataframe 2018

In [2]:
import rasterio

# Abrir el archivo raster
raster_path = r"D:\Cartografia\GEOCOL_2004\GEOCOLCTM12.tif"            #GEOCOL TIFF
raster = rasterio.open(raster_path)
dfCTM12=pd.read_csv(dir+"\\12.csv")

from rasterio.sample import sample_gen

# Crear una lista de coordenadas (X, Y)
coordinates = list(zip(dfCTM12["Este"], dfCTM12["Norte"]))

# Extraer valores del raster para cada coordenada
values = []
for coord in coordinates:
    try:
        # Convertir la coordenada a valor del raster
        value = list(raster.sample([coord]))[0][0]
    except IndexError:
        value = None  # Manejar valores fuera del rango del raster
    values.append(value)

# Añadir los valores al DataFrame
dfCTM12["RasterValue"] = values
output_path = dir+"\\n.csv"
dfCTM12.to_csv(output_path, index=False)

#### Dataframe 2018
import pandas as pd

d12=pd.read_csv(dir+"\\12.csv")
d84=pd.read_csv(dir+"\\84.csv")
dxyz=pd.read_csv(dir+"\\xyz.csv")


d12_s = d12[["ID", "Norte","Este"]]
d84_s = d84[["ID", "Latitud","Longitud","Altura_Elipsoidal"]]
dxyz_s = dxyz[["ID", "X","Y","Z"]]
dn_s=dfCTM12[["ID","RasterValue"]]

merged_df = (
    d84_s
    .merge(dxyz_s, on="ID", how="inner")
    .merge(d12_s, on="ID", how="inner")
    .merge(dn_s, on="ID", how="inner")
)

# Guardar el DataFrame combinado en un archivo CSV
output_path = dir + "\\data2018.csv"
df1["N"]=merged_df["RasterValue"]
merged_df.to_csv(output_path, index=False)
df1.to_csv(dir+"\\Data2024.csv", index=False)


---
### Archivo Geografico

In [3]:
import os 
import geopandas as gpd
from shapely.geometry import Point

Nombre=Nombre
dir9= dir.replace("8. Listado de coordenadas", "9. Archivo Geográfico")
df_Geo=pd.DataFrame()
df_Geo=merged_df[["ID","Norte","Este","Altura_Elipsoidal","RasterValue"]]
df_Geo["ALTURA ORTOMETRICA"]=df_Geo["Altura_Elipsoidal"]-df_Geo["RasterValue"]
df_Geo=df_Geo.drop(columns=["RasterValue"])
df_Geo = df_Geo.rename(columns={'ID': 'Nombre', 'Norte': 'Norte (m)', 'Este': 'Este (m)', 'Altura_Elipsoidal': 'Altura Elipsoidal'})



for archivo in os.listdir(dir9):
    ruta_archivo = os.path.join(dir9, archivo)
    if os.path.isfile(ruta_archivo):  # Verifica si es un archivo
        os.remove(ruta_archivo)  # Elimina el archivo

df_Geo.to_csv(dir9+Nombre+".csv",index=False)

df_Geo['geometry'] = df_Geo.apply(lambda row: Point(row['Este (m)'], row['Norte (m)']), axis=1)
gdf = gpd.GeoDataFrame(df_Geo, geometry='geometry')

# WKT 
crs_9377 = (
    "PROJCRS[\"MAGNA-SIRGAS 2018 / Origen-Nacional\","
    "    BASEGEOGCRS[\"MAGNA-SIRGAS 2018\","
    "        DATUM[\"Marco Geocentrico Nacional de Referencia 2018\","
    "            ELLIPSOID[\"GRS 1980\",6378137,298.257222101,"
    "                LENGTHUNIT[\"metre\",1]]],"
    "        PRIMEM[\"Greenwich\",0,"
    "            ANGLEUNIT[\"degree\",0.0174532925199433]],"
    "        ID[\"EPSG\",20046]],"
    "    CONVERSION[\"Colombia Transverse Mercator\","
    "        METHOD[\"Transverse Mercator\","
    "            ID[\"EPSG\",9807]],"
    "        PARAMETER[\"Latitude of natural origin\",4,"
    "            ANGLEUNIT[\"degree\",0.0174532925199433],"
    "            ID[\"EPSG\",8801]],"
    "        PARAMETER[\"Longitude of natural origin\",-73,"
    "            ANGLEUNIT[\"degree\",0.0174532925199433],"
    "            ID[\"EPSG\",8802]],"
    "        PARAMETER[\"Scale factor at natural origin\",0.9992,"
    "            SCALEUNIT[\"unity\",1],"
    "            ID[\"EPSG\",8805]],"
    "        PARAMETER[\"False easting\",5000000,"
    "            LENGTHUNIT[\"metre\",1],"
    "            ID[\"EPSG\",8806]],"
    "        PARAMETER[\"False northing\",2000000,"
    "            LENGTHUNIT[\"metre\",1],"
    "            ID[\"EPSG\",8807]]],"
    "    CS[Cartesian,2],"
    "        AXIS[\"northing (N)\",north,"
    "            ORDER[1],"
    "            LENGTHUNIT[\"metre\",1]],"
    "        AXIS[\"easting (E)\",east,"
    "            ORDER[2],"
    "            LENGTHUNIT[\"metre\",1]],"
    "    USAGE["
    "        SCOPE[\"Cadastre, topographic mapping.\"],"
    "        AREA[\"Colombia - onshore and offshore. Includes San Andres y Providencia, Malpelo Islands, Roncador Bank, Serrana Bank and Serranilla Bank.\"],"
    "        BBOX[-4.23,-84.77,15.51,-66.87]],"
    "    ID[\"EPSG\",9377]]"
)
gdf.set_crs(crs_9377, inplace=True)

# Guardar como Shapefile
shapefile_path = dir9+Nombre+".shp"
gdf.to_file(shapefile_path, driver='ESRI Shapefile')


#Eliminar CSV
for archivo in os.listdir(dir):
    if archivo.endswith(".csv"):
        archivo_path = os.path.join(dir, archivo)
        os.remove(archivo_path)

c:\ProgramData\anaconda3\envs\entorno37_20240321\lib\site-packages\ipykernel_launcher.py:70: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


---